In [1]:
import numpy as np
import pandas as pd
import random, string
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [2]:
!ls

gender_submission.csv  predictions.csv	test.csv  Titanic.ipynb  train.csv


In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_data, test_data = train_test_split(train_data, test_size=0.2)

train_data=train_data.set_index(train_data['PassengerId'])
test_data=test_data.set_index(test_data['PassengerId'])

train_data[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,,
882,882,0,3,"Markun, Mr. Johann",male,33.0,0,0,349257,7.8958,NaN,S
508,508,1,1,"Bradley, Mr. George (""George Arthur Brayton"")",male,NaN,0,0,111427,26.5500,NaN,S
464,464,0,2,"Milling, Mr. Jacob Christian",male,48.0,0,0,234360,13.0000,NaN,S
825,825,0,3,"Panula, Master. Urho Abraham",male,2.0,4,1,3101295,39.6875,NaN,S
447,447,1,2,"Mellinger, Miss. Madeleine Violet",female,13.0,0,1,250644,19.5000,NaN,S


In [5]:
mean = None
def prepare_both(X):
    X['Sex_i'] = X['Sex'].apply(lambda a: 1 if a == 'female' else 0)
    X['Age'] = X['Age'].fillna(29) # 29 is ~mean
    X['Embarked'] = X['Embarked'].fillna('A')
    X['Level'] = str(X['Cabin'].fillna('Z'))[0]
    
    s=['Sex_i', 'Age', 'SibSp', 'Parch']
    return pd.DataFrame(X[s], index=X.index)
        
def prepare_train(X):
    train_X = prepare_both(X)
    transformers=[]
    
    enc1 = OneHotEncoder()
    train_X = train_X.join( 
        pd.DataFrame(
            data=enc1.fit_transform(X[['Pclass']]).toarray(), index=X.index, 
            columns=enc1.get_feature_names()),
    )
    transformers.append(enc1)

    enc2 = OneHotEncoder()
    train_X = train_X.join( 
        pd.DataFrame(data=enc2.fit_transform(X[['Embarked']]).toarray(), index=X.index, 
                     columns=enc2.get_feature_names())
    )
    transformers.append(enc2)
    
    enc3 = OneHotEncoder()
    train_X = train_X.join( 
        pd.DataFrame(data=enc3.fit_transform(X[['Level']]).toarray(), index=X.index, 
                     columns=enc3.get_feature_names())
    )
    transformers.append(enc3)

    return train_X, transformers

def prepare_test(X, transformers):
    test_X = prepare_both(X)
    
    test_X = test_X.join( 
        pd.DataFrame(data=transformers[0].transform(X[['Pclass']]).toarray(), index=X.index,
                     columns=transformers[0].get_feature_names())
    )
    test_X = test_X.join( 
        pd.DataFrame(data=transformers[1].transform(X[['Embarked']]).toarray(), index=X.index,
                     columns=transformers[1].get_feature_names())
    )
    test_X = test_X.join( 
        pd.DataFrame(data=transformers[2].transform(X[['Level']]).toarray(), index=X.index,
                     columns=transformers[2].get_feature_names())
    )
    return test_X

X_train, transformers = prepare_train(train_data)
X_test = prepare_test(test_data, transformers)

Y_train = train_data['Survived']
Y_test = test_data['Survived']

X_train

,Sex_i,Age,SibSp,Parch,x0_1,x0_2,x0_3,x0_A,x0_C,x0_Q,x0_S,x0_P
PassengerId,,,,,,,,,,,,
882,0,33.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
508,0,29.0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
464,0,48.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
825,0,2.0,4,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
447,1,13.0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
331,1,29.0,2,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
414,0,29.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
617,0,34.0,1,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0


In [7]:
#X_test

In [ ]:
#import warnings
#warnings.filterwarnings("ignore")

In [19]:
def train(X_train, Y_train, design):
    model = MLPClassifier(solver='lbfgs', hidden_layer_sizes=design, max_iter=1000)
    model.fit(X_train, Y_train)
    return model 

def test_configs():
    max_test_acc=0
    max_design=-1
    for i in range(2,20):
        x=i*i*i
        design=(x,x,x)
        train_acc, test_acc = train(X_train, Y_train, X_test, Y_test, design)
        train_acc=model.score(X_train, Y_train)
        test_acc=model.score(X_test, Y_test)
    
        if test_acc > max_test_acc:
            max_test_acc=test_acc
            max_design=design
    
    print('Max_test_acc=' + str(max_test_acc) + ' Network design=' + str(max_design))

model = train(X_train, Y_train, design=(50, 50, 50))
train_acc=model.score(X_train, Y_train)
print('train_acc={}, '.format(str(train_acc)))

test_acc=model.score(X_test, Y_test)
print('test_acc={}, '.format(str(test_acc)))

train_acc=0.8258426966292135, 


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 12 is different from 13)

In [15]:
test_data = pd.read_csv('test.csv')
test_data=test_data.set_index(test_data['PassengerId'])
X_test = prepare_test(test_data, transformers)
X_test=X_test.join(pd.DataFrame(model.predict(X_test), index=X_test.index, columns=['Survived']))
X_test[['Survived']].to_csv('predictions.csv')